<img src="../images/logo.jpg" style="width:85px;height:85px;float:left" /><h1 style="position:relative;float:left;display:inline">DCGAN in Theory and Practice</h1>

<a href="https://colab.research.google.com/github/zurutech/gans-from-theory-to-production/blob/master/3.%20TFGAN/3.1.%20DCGAN%20in%20Theory%20and%20Practice.ipynb">
<img align="left" src="https://cdn-images-1.medium.com/max/800/1*ZpNn76K98snC9vDiIJ6Ldw.jpeg"></img>
</a>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Theory" data-toc-modified-id="Theory-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Theory</a></span><ul class="toc-item"><li><span><a href="#Enter-DCGAN" data-toc-modified-id="Enter-DCGAN-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Enter DCGAN</a></span></li><li><span><a href="#GANEstimator" data-toc-modified-id="GANEstimator-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>GANEstimator</a></span></li><li><span><a href="#Generator:-from-noise-to-insight" data-toc-modified-id="Generator:-from-noise-to-insight-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Generator: from noise to insight</a></span><ul class="toc-item"><li><span><a href="#Deconvolution" data-toc-modified-id="Deconvolution-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Deconvolution</a></span></li><li><span><a href="#Batch-Normalization" data-toc-modified-id="Batch-Normalization-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Batch Normalization</a></span></li><li><span><a href="#generator_fn()" data-toc-modified-id="generator_fn()-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>generator_fn()</a></span></li></ul></li><li><span><a href="#Discriminator" data-toc-modified-id="Discriminator-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Discriminator</a></span><ul class="toc-item"><li><span><a href="#discriminator_fn()" data-toc-modified-id="discriminator_fn()-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>discriminator_fn()</a></span></li></ul></li><li><span><a href="#Loss-function:-a-bridge-between-two-networks" data-toc-modified-id="Loss-function:-a-bridge-between-two-networks-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Loss function: a bridge between two networks</a></span></li></ul></li><li><span><a href="#Practice---Introduction" data-toc-modified-id="Practice---Introduction-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Practice - Introduction</a></span><ul class="toc-item"><li><span><a href="#Input-Functions" data-toc-modified-id="Input-Functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Input Functions</a></span><ul class="toc-item"><li><span><a href="#Loading-Images" data-toc-modified-id="Loading-Images-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Loading Images</a></span></li></ul></li><li><span><a href="#Training" data-toc-modified-id="Training-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#TensorBoard" data-toc-modified-id="TensorBoard-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>TensorBoard</a></span></li></ul></li><li><span><a href="#Predictions" data-toc-modified-id="Predictions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Predictions</a></span><ul class="toc-item"><li><span><a href="#predict_input_fn()" data-toc-modified-id="predict_input_fn()-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>predict_input_fn()</a></span></li></ul></li><li><span><a href="#Preparing-for-Production" data-toc-modified-id="Preparing-for-Production-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Preparing for Production</a></span><ul class="toc-item"><li><span><a href="#serving_input_receiver_fn" data-toc-modified-id="serving_input_receiver_fn-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>serving_input_receiver_fn</a></span></li><li><span><a href="#Exporting-the-model-for-production" data-toc-modified-id="Exporting-the-model-for-production-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Exporting the model for production</a></span></li><li><span><a href="#Local-CloudML-Predictions-testing" data-toc-modified-id="Local-CloudML-Predictions-testing-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Local CloudML Predictions testing</a></span><ul class="toc-item"><li><span><a href="#Generate-the-test-noise" data-toc-modified-id="Generate-the-test-noise-4.3.1"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Generate the test noise</a></span></li><li><span><a href="#Execute-the-test" data-toc-modified-id="Execute-the-test-4.3.2"><span class="toc-item-num">4.3.2&nbsp;&nbsp;</span>Execute the test</a></span></li></ul></li></ul></li><li><span><a href="#To-the-serving" data-toc-modified-id="To-the-serving-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>To the serving</a></span></li><li><span><a href="#NOTES" data-toc-modified-id="NOTES-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>NOTES</a></span></li><li><span><a href="#Links" data-toc-modified-id="Links-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Links</a></span></li></ul></div>

In [ ]:
! pip install tensorflow==1.13.1
#! pip install tensorflow-gpu==1.13.1
! pip install tensorflow-datasets

In [ ]:
import numpy as np
import tensorflow as tf

# Import tfgan from contrib
tfgan = tf.contrib.gan

## Theory

### Enter DCGAN

![DCGAN](images/dcgan.png)

If you take a look at [this impressive list of GANs [2\]](#2), you would find out that **DCGAN**, the architecture of our choice, is a small drop in the ocean. While there may be sexier ones, very few offers the same level of clarity, performance and computational efficiency. For these reasons DCGAN is considered one of the cornerstones of this field.

Alec Radford, Luke Metz and Soumith Chintala proposed the architecture in [their 2015 paper [3\]](#3). The idea behind DCGAN is quite straightforward. Combining a set of architectural constraints with the power of CNN yielded a robust, stable and competitive model. Moreover, the architecture is simple: 4 deconvolutional layers for the `Generator` and 4 convolutional layers for the `Discriminator`. The constraints are the following:

- All pooling layers are replaced with strided convolutions (discriminator) and fractionally-strided convolutions (generator).
- Batch-normalization used in both networks.
- Removal of the fully-connected layers (except for the discriminator output).
- `ReLU` for all Generator layers except the output, which uses `tanh`.
- `LeakyReLU` activation in the discriminator for all layers.

Recently there have been some advancements in state of the art (e.g., Spectral Normalization). However, it is essential to have a firm understanding of the basic concepts. So, we leave you (an opinionated) list of further resources at the end of this notebook, to get you up to speed with the most exciting researches.

### GANEstimator
We are going to create all that we need for the `GANEstimator` object. The signature is the following:

```python
__init__(
    model_dir=None,
    generator_fn=None,
    discriminator_fn=None,
    generator_loss_fn=None,
    discriminator_loss_fn=None,
    generator_optimizer=None,
    discriminator_optimizer=None,
    get_hooks_fn=None,
    get_eval_metric_ops_fn=None,
    add_summaries=None,
    use_loss_summaries=True,
    config=None,
    warm_start_from=None,
    is_chief=True
)
```
With this in mind, we are going to create the `generator_fn` and the `discriminator_fn`. These two functions represent the `Generator` and the `Discriminator`, respectively. 
Finally, we are going to set to the `GANEstimator` all the other parameters on the fly.

### Generator: from noise to insight

![DCGAN Generator](images/dcgan_generator.png)

Recalling the theoretical explanation, the Generator is the network responsible for the data-generation. It learns how to fool the discriminator, so it learns how to produce realistic results. Those results are then "sampled" from the learned manifold.

The most common type of generator input is noise (i.e.: random values). More specialized GANs may require extra parameters. Since our full-demo uses a Deep Convolutional GAN (DCGAN), we don't need any other parameters.

#### Deconvolution

Intuitively, the idea behind this operation is the following:

![Deconvolution](images/deconvolution.png)


> When neural networks generate images, they build the images starting from low-resolution high-level descriptions. In this way, the network starts describing a 
> rough representation and then fill in the details to create the final image.
>
>To do this, we need some way to go from a lower resolution image to a higher one. We generally do this with the deconvolution operation. 
>Roughly, deconvolution layers allow the model to use the points from the small image to “paint” a larger area in the bigger output image.

In practice, the deconvolution operation is often implemented by resizing, using bi-linear or nearest neighbor interpolation, followed by a convolution operation.


#### Batch Normalization

What you need to know about batch normalization is that is a layer that normalizes the values. TensorFlow makes it very easy to implement such operation:  

```python
tf.keras.layers.BatchNormalization()
```

The benefit of using batch normalization has been extensively discussed and proved in various papers. We do not enter into the theoretical depth of BatchNormalization, but you can refer to [[3\]](#3), [[8\]](#8) and [[9\]](#9) to learn more about the topic.

#### generator_fn()

In [ ]:
def deconv2d(inputs, filters, strides=(1, 1), activation=tf.nn.relu):
    """\"Deconvolution\" layer.
    
    It uses upsampling with nearest neighbor interpolation to reduce the
    presence of checkerboard artifacts.
    """
    
    input_h, input_w = inputs.shape[1].value, inputs.shape[2].value
    layer_1 = tf.image.resize_nearest_neighbor(
        inputs, (2 * input_h, 2 * input_w), name="NNUpSample2D"
    )
    # Padding before convolution is used to reduce boundary artifacts
    layer_1 = tf.pad(layer_1, [[0, 0], [2, 2], [2, 2], [0, 0]], mode="CONSTANT")
    layer_2 = tf.layers.conv2d(
        inputs=layer_1,
        filters=filters,
        kernel_size=5,
        padding="valid",
        use_bias=False,
        activation=activation,
        strides=strides,
    )
    return layer_2

def generator_fn(inputs, mode):
    """Generator producing images from noise.

        Args:
            inputs: A single Tensor representing noise.
            mode: tf.estimator.ModeKeys

        Returns:
            A 64x64 (None, 4096) flattened tensor whose values are
            inside the (-1, 1) range."""
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    linear = tf.layers.dense(inputs=inputs, units=1024 * 4 * 4, activation=tf.nn.relu)
    net = tf.reshape(linear, (-1, 4, 4, 1024))
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 512)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 256)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 128)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 64)
    net = tf.layers.conv2d(
        inputs=net,
        filters=3,
        kernel_size=5,
        padding="same",
        data_format="channels_last",
        use_bias=False,
        strides=(1, 1),)
    output = tf.tanh(net)
    return output

### Discriminator

![DCGAN Discriminator](images/dcgan_discriminator.png)

As for the `input_fn()` we can reuse all the code we defined for the vanilla `Estimator` describing the discriminator network architecture.

#### discriminator_fn()

In [ ]:
def custom_conv2d(inputs, filters, strides=(2,2)):
    """Helper layer used to instantiate `tf.layers.conv2d` with proper arguments."""
    layer_1 = tf.layers.conv2d(
        inputs=inputs,
        filters=filters,
        kernel_size=5,
        padding="same",
        data_format="channels_last",
        use_bias=False,
        strides=strides,
    )
    layer_1 = tf.nn.leaky_relu(layer_1, alpha=0.2)

    return layer_1

def discriminator_fn(inputs, conditioning, mode):
    """Build the Discriminator network.
    Args:
        inputs: a batch of images to classify, expected input shape (None, 64, 64 , 3)
        conditioning: a batch of labels, it is used for conditioning in the some model (es Conditional GAN).
            GANEstimator wants this parameters around, just define an arguments so that discriminator_fn is not broken.
        mode: tf.estimator.ModeKey
    
    Returns:
            The output (logits) of the discriminator.
    """
    
    # In every mode, define the model
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    net = custom_conv2d(inputs, filters=64) # <--64*64*3
    net = custom_conv2d(net, filters=128) # <-- 32*32*64   
    net = tf.layers.batch_normalization(net, training=is_training)
    net = custom_conv2d(net, filters=256) # <-- 16*16*128
    net = tf.layers.batch_normalization(net, training=is_training)
    net = custom_conv2d(net, filters=512) # <-- 8*8*256
    net = tf.layers.batch_normalization(net, training=is_training)
    net = custom_conv2d(net, filters=1024) # <-- 4*4*512
    net = tf.layers.batch_normalization(net, training=is_training)
    net = tf.reshape(net, (-1, net.shape[1] * net.shape[2] * net.shape[3]))
    output = tf.layers.dense(net, units=1) # <-- 2*2*1024 --> 1

    return output

### Loss function: a bridge between two networks

There are a lot of loss functions usable in GANs' architectures. From very domain-specific ones to others that are perfect for general use cases. Since our goal here falls in the latter category, we use the so-called **Non-Saturating Loss** which is the non-saturating variant of the **MinMax Loss** proposed by Goodfellow in the [original paper [1\]](#1).

As stated above, one of `TFGAN`'s beauties is its offer of ready-to-use losses. If you cannot find the loss you want, you can create your own. 

We use the following two losses:

```python
generator_loss_fn=tfgan.losses.minimax_generator_loss

discriminator_loss_fn=tfgan.losses.minimax_discriminator_loss
```

It is that easy.

## Practice - Introduction

Having seen the general structure of the TFGAN library, we dive right into the model architecture.

In our showcasing of the TensorFlow API, we have built an image recognition network. GANs, however, need the training of both a Generator and a Discriminator together.

This task, in the case of writing it in *vanilla TensorFlow*, turns out to be verbose and performance-constrained.

On the other hand, in the case of *Estimator API* results in somewhat complicated.

Thanks to *GANEstimator*, instead, it becomes remarkably simple.

### Input Functions
A `GANEstimator` object it is used to train the GAN models. The `GANEstimator` `train` function takes as a parameter an "input function" (i.e.: `input_fn`). We have to write that `input_fn` in a specific manner. The way we need to write that function is pretty much the same as the one we would define for a standard `tf.Estimator`. The most important thing is its *return value* that should be one of the following:

> A `tf.data.Dataset` object: Outputs of Dataset object must be a tuple (features, labels) with the same constraints as below.
> 
> A tuple (features, labels): Where `features` is a Tensor or a dictionary of string feature name to Tensor and `labels` is a Tensor or a dictionary of string label name to Tensor. [...]

However, GANs training (and evaluating) needs to use two types of features: the fake ones (noise) and the real ones (real images). On the other hand, as we have just said, we need to be compatible with the standard `tf.Estimator` return "signature", i.e.: (features, labels). Exploiting this return signature, we can achieve our goal. Hence, instead of returning a (features, labels) couple we return a (noise, real) couple. In practice, what we need to do is the following:

1. Structure the `input_fn` code as a **function object** as requested by `GANEstimator.train` (and `.evaluate`).
2. Return **noise** first and then the **real data** (the images) instead of (features, labels) couple. Optionally we can return the labels too.

During the training, the `GANEstimator` object will internally manage the training process and automatically pass the needed values to the `generator_fn` and `discriminator_fn` functions.

#### Loading Images

We use the Celeb-A dataset downloaded using tensorflow-datasets (tfds) to easily create the input function that satisfies all the requirements.

![DCGAN Generator](images/celeba_sample_imgs.png)

The image is an example of the CelebA Dataset, source: [here](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html). 

In [ ]:
import tensorflow_datasets as tfds

def get_train_input_fn(batch_size, num_epochs, noise_dim):
    """The input function that builds the `tf.data.Dataset` object and instantiate
    the iterator correctly ready to be use.
    Returns:
        the iterator associated to the built Dataset object.
    """
    
    def convert_and_resize(features):
        image = tf.image.convert_image_dtype(features["image"], dtype=tf.float32)
        image = (image - 0.5) * 2
        image = tf.image.resize(image, size=(64, 64))
        features["image"] = image
        return features

    def _input_fn():
        real_data = tfds.load("celeb_a",split=tfds.Split.ALL)
        real_data = real_data.map(convert_and_resize).map(lambda feature: feature["image"])
        real_data = real_data.batch(batch_size, drop_remainder=True).repeat(num_epochs)
        real_data_iterator = real_data.make_one_shot_iterator()
        
        noise = tf.random_normal([batch_size, noise_dim], name="train_noise")
        real_batch = real_data_iterator.get_next()
        real_batch.set_shape((batch_size,) + tuple(real_batch.shape[1:]))
        return noise, real_batch
    return _input_fn

### Training

In [ ]:
def dcgan():
    
    # Hyperparameters
    model_dir = "../logs/"
    batch_size = 128
    num_epochs = 1
    noise_dim = 100
    
    # Run Configuration (it has other arguments)
    run_config = tf.estimator.RunConfig(
        model_dir=model_dir, save_summary_steps=50, save_checkpoints_steps=500)
    
    # Instantiate the GANEstimator object
    gan_estimator = tfgan.estimator.GANEstimator(
        config=run_config,
        generator_fn=generator_fn,
        discriminator_fn=discriminator_fn,
        generator_loss_fn=tfgan.losses.modified_generator_loss,
        discriminator_loss_fn=tfgan.losses.modified_discriminator_loss,
        generator_optimizer=tf.train.AdamOptimizer(0.0002, 0.5),
        discriminator_optimizer=tf.train.AdamOptimizer(0.0002, 0.5),
        add_summaries=tfgan.estimator.SummaryType.IMAGES
    )
    
    # Instantiate the train_input_fn
    # The model will train until it exhausts the Dataset which is repeated EPOCH times
    train_input_fn = get_train_input_fn(batch_size, num_epochs, noise_dim)
    trained_model = gan_estimator.train(train_input_fn, max_steps=None)
    return trained_model

### TensorBoard

In order to track our training we need to launch a **TensorBoard** session pointing to the folder (`model_dir`) containing the logs generated by our training.

```bash
tensorboard --logdir=PATH_TO_YOUR_MODEL_DIR
```

In [ ]:
trained_model = dcgan()

## Predictions

We train Machine Learning models because we want them to perform some specific task since we now have our GAN trained, we can finally use it to "predict" AKA generate a new image from a noise vector.

Once again, TFGAN makes it as easy as invoking the `predict()`  method of our `GANEstimator` while passing to it a `predict_input_fn` as a required argument. 

### predict_input_fn()

As previously mentioned while theoretically identical, `train_input_fn` and `predict_nput_fn()` should be implemented differently.  The first one is a simple `tf.random_normal()` node, the second should make proper use of the `tf.Dataset` API.

In [ ]:
def _predict_input_fn(batch_size, noise_dims=100, **kwargs):
    
    def predict_input_fn():
        noise_gen = np.array([np.float32(np.random.normal(size=[1, noise_dims])) for i in range(batch_size)])
        dataset = tf.data.Dataset.from_tensor_slices(noise_gen)
        dataset = dataset.batch(batch_size)
        iterator = dataset.make_one_shot_iterator()
        return iterator.get_next()
        
    return predict_input_fn

In [ ]:
predictions_batch = 3
predict_input_fn = _predict_input_fn(batch_size=predictions_batch)
predictions = trained_model.predict(predict_input_fn)
[next(predictions) for _ in range(predictions_batch)]

## Preparing for Production

Now that we have our model trained and ready we are just a couple steps away from being able to put our model into production.

### serving_input_receiver_fn

To serve a model in production, we first need to equip it with an interface which will receive data from our requests, such interface is specified by the aptly named `serving_input_receiver_fn`. Of the three input functions, this is the trickiest one since it has its peculiar API.

This functions requires its output to be a either a `ServingInputReceiver` or a `TensorServingInputReceiver` object; the documentation on their use is clear:

> The normal `ServingInputReceiver` always returns a feature dict, even if it contains only one entry, and so can be used only with models that accept such a dict. 
>For models that accept only a single raw feature, the `serving_input_receiver_fn` provided to `Estimator.export_savedmodel()` should return this `TensorServingInputReceiver`.

Since our model needs only a noise vector to get going, we can use `TensorServingInputReceiver`.

>The expected return values are: 
> - **features**: A single `Tensor` or `SparseTensor`, representing the feature to be passed to the model. 
> - **receiver_tensors**: A Tensor, SparseTensor, or dict of string to Tensor or SparseTensor, specifying input nodes where this receiver expects to be fed by default. Typically, this is a single placeholder expecting serialized `tf.Example` protos. 
> - **receiver_tensors_alternatives**: a dict of string to additional groups of receiver tensors, each of which may be a `Tensor`, `SparseTensor`, or dict of string to `Tensor` or `SparseTensor`. These named receiver tensor alternatives generate additional serving signatures, which may be used to feed inputs at different points within the input receiver subgraph. A typical usage is to allow feeding raw feature Tensors downstream of the `tf.parse_example()` op. Defaults to None.

In [ ]:
def _serving_input_receiver_fn():
    """Instantiate a placeholder for our serving input data.

    Call the custom function `serving_input_fn()`, built following
    TensorFlow Estimator API convention, initializing the placeholder for
    the noise we will feed the model during its serving.

    The Serving Input function has two key elements:

        - the data-processing step, where we concretely prepare data to be
        fed to the:
        - Placeholder, it is the node where the input are fed.

    The things to notice is that while using `ServingInputReceiver`
    your data processing step should have at its core the parsing of the
    tf.Example received.

    With `TensorServingInputReceiver` our data won't really be passed by the
    request, instead it will have to be generated ''model-side'' inside the
    `_serving_input_receiver_fn` itself.

    Returns:
        tf.estimator.export.TensorServingInputReceiver passing the
        placeholder for the noise to it.
    """

    receiver_tensors = tf.placeholder(shape=[None, 100], dtype=tf.float32, name="serving_noise")
    return tf.estimator.export.TensorServingInputReceiver(receiver_tensors, receiver_tensors)

### Exporting the model for production

Exporting the model is as easy as calling `GANEstimator.export_savedmodel()` which the same as the normal `Estimator`.

> This method builds a new graph by first calling the serving_input_receiver_fn to obtain feature Tensors, and then calling this Estimator's model_fn to generate the model graph based on those features. It restores the given checkpoint (or, lacking that, the most recent checkpoint) into this graph in a fresh session. Finally it creates a timestamped export directory below the given export_dir_base, and writes a SavedModel into it containing a single MetaGraphDef saved from this session.

We have to specify an output folder and the `serving_input_receiver_fn`.

In [ ]:
export_dir_base = "../assets/exported_models"
trained_model.export_savedmodel(
    export_dir_base, _serving_input_receiver_fn)

### Local CloudML Predictions testing

Before going over the required steps for model serving, we want to test it locally to make sure that the exported models will behave correctly once loaded onto CloudML Engine.

#### Generate the test noise

Google Cloud often uses Newline Delimited JSON  when working with JSON-formatted data; we can use the `jsonlines` Python library assure our compliance with the standard.

In [ ]:
import jsonlines

In [ ]:
BATCH_SIZE = 2
NOISE_DIMS = 100

with jsonlines.open("../assets/test_noise.ndjson", "w") as writer:
    noise = np.random.normal(size=(BATCH_SIZE, NOISE_DIMS)).tolist()
    noised_dict = [{"input": n} for n in noise]
    writer.write_all(noised_dict)

#### Execute the test

In [ ]:
%%bash

MODEL_DIR="../assets/exported_models"
MODEL_ID="1535124551"
JSON_INSTANCES="../assets/test_noise.ndjson"

rm ~/google-cloud-sdk/lib/googlecloudsdk/command_lib/ml_engine/*.pyc

gcloud ml-engine local predict \
    --model-dir=$MODEL_DIR/$MODEL_ID \
    --json-instances=$JSON_INSTANCES

## To the serving

Now that we have the exported model we are ready to finally serve our model.

## NOTES

[1]: [The GAN Landscape: Losses, Architectures, Regularization, and Normalization](https://arxiv.org/abs/1807.04720v1), which in our opinion is one of the most thorough scientific studies on GANs out there, suggests that `Spectral Normalization` is the real big deal and that `BatchNorm` actually hurt performance if applied to the Discriminator. We still decided to go with the classic formulation of DCGAN as not to overtax you with theoretical discussions. We leave the implementation of the `Spectral Normalization`(and SNGAN) to you as an exercise. On the theoretical treating of `BatchNorm` we recommend a back to back reading of [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167), [Understanding Batch Normalization](https://arxiv.org/abs/1806.02375v1), [How Does Batch Normalization Help Optimization? (No, It Is Not About Internal Covariate Shift)](https://arxiv.org/abs/1805.11604) 

[2]: While often you want to work with **features (and labels)** fetched from a `tf.Dataset`, the **noise** should always be instantiated using a simple TensorFlow node. Trying to create a noise-containing `tf.Dataset` for the `train_input_fn` is not worth the effort. NOTE: that as things are different for the [predict_input_fn](##predict_input_fn()). 

[3]: See [Tensorflow 2.0: Keras is not (yet) a simplified interface to Tensorflow[10\]](#10) to have an understanding on the compatibility between Keras and Tensforflow (2.0).


## Links

<a id="1">[1]</a>: Generative Adversarial Networks : https://arxiv.org/abs/1406.2661

<a id="2">[2]</a>: really-awesome-gan : https://github.com/nightrome/really-awesome-gan

<a id="3">[3]</a>: DCGAN : https://arxiv.org/abs/1511.06434

<a id="4">[4]</a>: Deconvolution and Checkerboard Artifacts : https://distill.pub/2016/deconv-checkerboard/

<a id="5">[5]</a>: TFGAN MNIST GAN Example : https://github.com/tensorflow/models/tree/master/research/gan/mnist_estimator

<a id="6">[6]</a>: Estimator API : https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator

<a id="7">[7]</a>: export_savedmodel : https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator#export_savedmodel

<a id="8">[8]</a>: Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift : https://arxiv.org/abs/1502.03167

<a id="9">[9]</a>: How Does Batch Normalization Help Optimization? : https://arxiv.org/abs/1805.11604

<a id="9">[10]</a>: Tensorflow 2.0: Keras is not (yet) a simplified interface to Tensorflow : https://pgaleone.eu/tensorflow/keras/2019/01/19/keras-not-yet-interface-to-tensorflow/